# Desafio 07

TNT

In [1]:
import pandas as pd
import numpy as np

# Carregando banco

In [2]:
dataset = pd.read_csv('tnt_database.csv')

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9979 entries, 0 to 9978
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Açaí-Guaraná  9979 non-null   int64  
 1   Citrus        9979 non-null   int64  
 2   Estação       9979 non-null   object 
 3   LAT           9979 non-null   float64
 4   LONG          9979 non-null   float64
 5   Maçã-Verde    9979 non-null   int64  
 6   Movimentação  9979 non-null   int64  
 7   Original_269  9979 non-null   int64  
 8   Original_473  9979 non-null   int64  
 9   Pêssego       9979 non-null   int64  
 10  TARGET        9979 non-null   object 
 11  Tangerina     9979 non-null   int64  
 12  Tempo         9979 non-null   object 
 13  Zero          9979 non-null   int64  
 14  _id           9979 non-null   object 
 15  _rev          9979 non-null   object 
 16  row           9979 non-null   int64  
dtypes: float64(2), int64(10), object(5)
memory usage: 1.3+ MB


In [4]:
dataset.head()

,Açaí-Guaraná,Citrus,Estação,LAT,LONG,Maçã-Verde,Movimentação,Original_269,Original_473,Pêssego,TARGET,Tangerina,Tempo,Zero,_id,_rev,row
0,26,28,Pinheiros,-23.5664,-46.7030,43,78075,24,36,28,NORMAL,24,2019-6-23,13,0120472f15d821f24f91c6cf6701d88b,1-aa58175da90a12307d3ed70bdc902c2c,9173
1,41,9,Pinheiros,-23.5664,-46.7030,40,79432,53,78,41,NORMAL,38,2019-3-12,21,0120472f15d821f24f91c6cf670233ef,1-da709a8b94674992afdd9df7976f161d,9072
2,4,19,Butantã,-23.5844,-46.7252,25,67775,62,21,35,NORMAL,15,2018-4-7,51,0120472f15d821f24f91c6cf6704a6de,1-0ad2377710ecd310f8c810bfb2b04825,9457
3,19,43,Pinheiros,-23.5664,-46.7030,35,76773,29,61,18,NORMAL,37,2019-9-23,50,0120472f15d821f24f91c6cf67102d3a,1-c6e56500b4ad6860fa2396bd8453a7ba,9263
4,6,16,Pinheiros,-23.5664,-46.7030,26,77463,8,32,22,REABASTECER,10,2019-11-21,43,0120472f15d821f24f91c6cf6714cf5e,1-8cb748d7d25cb718673ce9ab4f5376f2,9321


In [25]:
dataset2 = dataset.drop(['LAT', 'LONG', '_id', '_rev', 'row', 'Estação', 'Tempo', 'Movimentação'], axis=1)


In [26]:
dataset2.head()

,Açaí-Guaraná,Citrus,Maçã-Verde,Original_269,Original_473,Pêssego,TARGET,Tangerina,Zero
0,26,28,43,24,36,28,NORMAL,24,13
1,41,9,40,53,78,41,NORMAL,38,21
2,4,19,25,62,21,35,NORMAL,15,51
3,19,43,35,29,61,18,NORMAL,37,50
4,6,16,26,8,32,22,REABASTECER,10,43


# Separando banco de treino e de teste

In [27]:
from sklearn.model_selection import StratifiedShuffleSplit

In [28]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

In [29]:
for train_index, test_index in split.split(dataset2, dataset2.TARGET):
    data_train = dataset2.loc[train_index]    
    data_val = dataset2.loc[test_index]

In [30]:

print(data_train.shape)
print(data_val.shape)

(7983, 9)
(1996, 9)


## Seperando as features e target

In [31]:
x_train = data_train.drop(['TARGET'], axis=1)
y_train = data_train.TARGET

In [35]:
x_val = data_val.drop(['TARGET'], axis=1)
y_val = data_val.TARGET

In [64]:
x_train.head()

,Açaí-Guaraná,Citrus,Maçã-Verde,Original_269,Original_473,Pêssego,Tangerina,Zero
7415,33,28,32,3,17,43,17,49
2547,6,40,43,17,13,37,16,38
3255,26,13,6,31,63,39,41,21
9320,43,7,10,25,58,15,34,65
4149,10,19,21,25,4,21,15,51


# Treinamento 

In [37]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier

In [125]:
clf = RandomForestClassifier(max_depth=10)

In [131]:
clf.fit(x_train, y_train)

pred_train = clf.predict(x_train)

In [132]:
print(classification_report(pred_train, y_train))

              precision    recall  f1-score   support

      NORMAL       0.97      0.99      0.98      5326
 REABASTECER       0.97      0.94      0.95      2657

    accuracy                           0.97      7983
   macro avg       0.97      0.96      0.97      7983
weighted avg       0.97      0.97      0.97      7983



# Validação

In [133]:
pred_val = clf.predict(x_val)

In [134]:
print(classification_report(y_val, pred_val))

              precision    recall  f1-score   support

      NORMAL       0.93      0.84      0.88      1356
 REABASTECER       0.72      0.86      0.78       640

    accuracy                           0.85      1996
   macro avg       0.82      0.85      0.83      1996
weighted avg       0.86      0.85      0.85      1996



In [135]:
print(accuracy_score(y_val, pred_val))

0.8471943887775552


## Insights

In [92]:
from sklearn.feature_selection import SelectKBest

In [93]:
model = RandomForestClassifier(max_depth=10)

for i in range(x_train.shape[1], 2, -1):
    skb = SelectKBest(k = i)
    xx = skb.fit_transform(x_train, y_train)
    model.fit(xx, y_train)
    pred = model.predict(skb.transform(x_val))
    print(i, accuracy_score(y_val, pred))

8 0.845691382765531
7 0.814128256513026
6 0.7955911823647295
5 0.7805611222444889
4 0.7540080160320641
3 0.75


In [94]:

for i in range(1,30, 2):
    clf2 = RandomForestClassifier(max_depth=i)
    clf2.fit(x_train, y_train)
    pred = clf2.predict(x_val)
    print(i, accuracy_score(y_val, pred))

1 0.6788577154308617
3 0.6963927855711423
5 0.7269539078156313
7 0.7805611222444889
9 0.8351703406813628
11 0.844188376753507
13 0.8532064128256514
15 0.8567134268537074
17 0.8547094188376754
19 0.8542084168336673
21 0.8562124248496994
23 0.844689378757515
25 0.8461923847695391
27 0.8552104208416834
29 0.843186372745491


# TESTE - RESULTADO 

In [137]:
data_test = pd.read_csv('to_be_scored.csv')


In [139]:
data_test.head()

,Tempo,Estação,LAT,LONG,Movimentação,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego
0,2018-11-30,Moema,-23.5489,-46.6388,17533,22,65,22,14,3,26,43,18
1,2018-12-1,Moema,-23.5489,-46.6388,18412,11,56,16,14,43,20,37,15
2,2018-12-2,Moema,-23.5489,-46.6388,19600,86,51,8,11,41,20,35,12
3,2018-12-3,Moema,-23.5489,-46.6388,20303,76,47,65,11,37,14,32,10
4,2018-12-4,Moema,-23.5489,-46.6388,17085,68,42,60,6,35,13,26,5


In [145]:
x_test = data_test.drop(['LAT', 'LONG', 'Estação', 'Tempo', 'Movimentação'], axis=1)
x_test.head()

,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego
0,22,65,22,14,3,26,43,18
1,11,56,16,14,43,20,37,15
2,86,51,8,11,41,20,35,12
3,76,47,65,11,37,14,32,10
4,68,42,60,6,35,13,26,5


In [146]:
y_pred = clf.predict(x_test)
result = x_test
result['target'] = y_pred
result.head()

,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,target
0,22,65,22,14,3,26,43,18,REABASTECER
1,11,56,16,14,43,20,37,15,NORMAL
2,86,51,8,11,41,20,35,12,NORMAL
3,76,47,65,11,37,14,32,10,NORMAL
4,68,42,60,6,35,13,26,5,REABASTECER


In [147]:
result.to_csv('results.csv', index=False)

In [148]:
result.head()

,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,target
0,22,65,22,14,3,26,43,18,REABASTECER
1,11,56,16,14,43,20,37,15,NORMAL
2,86,51,8,11,41,20,35,12,NORMAL
3,76,47,65,11,37,14,32,10,NORMAL
4,68,42,60,6,35,13,26,5,REABASTECER
